# Training and evaluation  

Das Programm in diesem Notebook dient dem Training und der Auswertung von neuronalen Netzen mit detectron2. Über die einzelnen Codeblöcke werden die nötigen Dateien geladen und das Modell entsprechend konfiguriert.

In [1]:
from detectron2.data.datasets import load_coco_json, register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

from detectron2.evaluation import COCOEvaluator, inference_on_dataset


import matplotlib.pyplot as plt
import random, cv2, time, os, shutil, ownlabelme2COCO, torch

In [ ]:
print(torch.__version__)

Definition des Hauptpfads, der Input-Pfade und der Output-Pfade

In [ ]:
WORK_DIR = "/home/julius/PowerFolders/Masterarbeit/"
os.chdir(WORK_DIR)

DATASET_PATH = "./1_Datensaetze/personData200/"
train_set_path = DATASET_PATH + "train_split/"
test_set_path = DATASET_PATH + "test_split/"

starttime = time.strftime("%d,%m,%Y-%H,%M")
model_path = "./trained_models/detectron2/{}/{}/".format(DATASET_PATH.split("/")[-2], starttime)

Wichtig für das Training mit detectron2 ist, dass die Daten bereits in die Bereiche "Training" und "Testing" unterteilt sind und für beide eine entsprechende '.json' Datei im von detectron2 unterstützten COCO Format vorliegt. Sollte dies nicht der Fall sein unterteilt der folgende Codeblock die Bilddaten in ein gegebenens Größenverhältnis.

In [ ]:
SPLIT = 5

if not os.path.isdir(train_set_path) & os.path.isdir(test_set_path):
    os.mkdir(train_set_path)
    os.mkdir(test_set_path)

    images = sorted([element for element in os.listdir(DATASET_PATH) if element.lower().endswith(".jpg")])
    jsons = sorted([element for element in os.listdir(DATASET_PATH) if element.endswith(".json")])

    if len(images) != len(jsons):
        break

    for count in range(len(images)):
        if count % SPLIT == 0:
            shutil.move(DATASET_PATH + images[count], test_set_path + images[count])
            shutil.move(DATASET_PATH + jsons[count], test_set_path + jsons[count])
        else:
            shutil.move(DATASET_PATH + images[count], train_set_path + images[count])
            shutil.move(DATASET_PATH + jsons[count], train_set_path + jsons[count])
    
    ownlabelme2COCO.main(test_set_path)
    ownlabelme2COCO.main(train_set_path)

Im Anschluss werden der Datensatz für das Training und dast Testing eingelesen

In [ ]:
load_coco_json(train_set_path + "COCO_json/output.json", train_set_path, "train_set")
register_coco_instances("train_set", {}, train_set_path + "COCO_json/output.json", train_set_path)
train_set_metadata = MetadataCatalog.get("train_set")
train_set_data = DatasetCatalog.get("train_set")

load_coco_json(test_set_path + "COCO_json/output.json", test_set_path, "test_set")
register_coco_instances("test_set", {}, test_set_path + "COCO_json/output.json", test_set_path)
test_set_metadata = MetadataCatalog.get("test_set")
test_set_data = DatasetCatalog.get("test_set")

Zur Kontrolle, ob alle Daten korrekt eingelesen wurden, kann mit dem folgenden Codeblock eine kleine Stichprobe angezeigt werden.

In [ ]:
random_image = random.sample(train_set_data, 1)[0]
image = cv2.imread(random_image["file_name"])
visualizer = Visualizer(image, metadata=train_set_metadata, scale=1, instance_mode=ColorMode.SEGMENTATION)
visualization = visualizer.draw_dataset_dict(random_image)
plt.figure(figsize=(25, 25))
plt.imshow(visualization.get_image()[:,:, ::-1])
plt.axis("off")

Die hauptsächlichen Stellschrauben für das Training in detectron2 werden über die Wahl der Modellarchitektur und die Konfigurationsdatei eingestellt. Ähnlich wie bei tensorflow basiert die Wahl der einzelnen Faktoren auf Erfahrung und dem Ausprobieren von unterschiedlichen Kombinationen aus Werten.

In [4]:
config = get_cfg()
config.merge_from_file("/home/julius/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

config.DATASETS.TRAIN = ("train_set",)
config.DATASETS.TEST = ("test_set",)
#config.DATALOADER.NUM_WORKERS = 2
config.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
config.OUTPUT_DIR = model_path
config.SOLVER.IMS_PER_BATCH = 2
config.SOLVER.REFERENCE_WORLD_SIZE = 1
config.SOLVER.BASE_LR = 0.02
config.SOLVER.MAX_ITER = 300
config.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
config.MODEL.RETINANET.NUM_CLASSES = 11
config.MODEL.ROI_HEADS.NUM_CLASSES = 11
config.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

'\nconfig.DATASETS.TRAIN = ("train_set",)\nconfig.DATASETS.TEST = ("test_set",)\nconfig.DATALOADER.NUM_WORKERS = 2\nconfig.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"\nconfig.OUTPUT_DIR = model_path\nconfig.SOLVER.IMS_PER_BATCH = 2\nconfig.SOLVER.REFERENCE_WORLD_SIZE = 1\nconfig.SOLVER.BASE_LR = 0.02\nconfig.SOLVER.MAX_ITER = 150\nconfig.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (128)\nconfig.MODEL.ROI_HEADS.NUM_CLASSES = 16\nconfig.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5'

Das eigentliche Training und die Evaluation des Modells wird über die folgenden Codezeilen initialisiert.

In [ ]:
torch.cuda.empty_cache()
trainer = DefaultTrainer(config)
trainer.resume_or_load(resume=False)
trainer.train()

torch.cuda.empty_cache()
evaluator = COCOEvaluator("test_set", config, distributed=False, output_dir=evaluation_path, use_fast_impl=False)
test_loader = build_detection_test_loader(config, "test_set")
inference_on_dataset(trainer.model, test_loader, evaluator)

Zur Dokumentation des Trainings und zur weiteren Verwendung der trainierten Modelle wird abschließend auch die Konfigurationsdatei exportiert.  
Es bleibt festzuhalten, dass das hier beschriebene Training nur einen kleinen Umfang besitzt. Die Trainingsergebnisse könnten wie im Umgang mit tensorflow mit Graphen visualisiert werden und die Konfigurationen müssten über verschiedene Trainingsdurchläufe diversifiziert werden. Es war im Rahmen dieser Arbeit nicht möglich vollumfängliche Trainings über große Epochenzahlen auf verschiedenen Datensätzen durchzuführen. Dafür sind die erzielten Ergebnisse vielversprechend und legen nahe, dass eine  weitere Beschäftigung mit der Objekterkennung auf Plakaten lohnenswert sein kann.

In [ ]:
config_dump = config.dump()
with open(model_path + "config.yaml", "w+") as output_file:
    output_file.write(config_dump)